In [22]:
from keras.models import load_model, Model, Sequential
from packages.Utility.CNN.FeatureManager import FeatureExtractor, FeatureReader, FeatureWriter
from packages.Utility.Gabor import GaborFilterBanks, GaborParams, RotatedGaborParams
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Layer, Dense
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.CNN.FeatureManager import FeatureExtractor, FeatureReader, FeatureWriter
from packages.Utility.Metrics import ConfusionMatrix, CNN_Evaluator, SVM_Evaluator
from sklearn import svm

# Inisialisasi

In [25]:
dataset_dir = 'dataset/bin/DG321'
weight_dir = 'trained/digit LN[1]/Fold 1/acc 0.9627 - epoch 122.h5'

# Load Dataset

In [26]:
reader = DatasetReader(dataset_dir)
fold = 0

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold)

# Mendefinisikan Model

In [30]:
model = Sequential()

c1 = Conv2D(name='c1', filters = 6,
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu', 
            input_shape = (32, 32, 1))
c2 = Conv2D(name='c2', filters = 16,
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu')
m1 = MaxPool2D(pool_size = 2)
m2 = MaxPool2D(pool_size = 2)
f1 = Flatten(name = 'flatten')
model.add(c1)
model.add(m1)
model.add(c2)
model.add(m2)
model.add(f1)
model.add(Dense(120, activation = 'sigmoid', name = 'd1'))
model.add(Dense(84, activation = 'sigmoid', name = 'd2'))
model.add(Dense(10, activation = 'softmax', name = 'output'))

# Load Weights

In [31]:
model.load_weights(weight_dir)

# Ekstraksi Fitur

In [32]:
extractor = FeatureExtractor(model, output_layer = 'flatten')
feature_train = extractor.extract(X_train)
feature_test = extractor.extract(X_test)

# Pelatihan SVM

In [36]:
cls = svm.SVC(kernel = 'linear', max_iter = 50000, C = 100, decision_function_shape='ovo', verbose = True)
cls.fit(feature_train, Y_train_single)

c:\python\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=50000, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

# Evaluasi Model
### Evaluasi untuk CNN

In [34]:
ev_cnn = CNN_Evaluator(model = model, labels = reader.labels)
y_true_train_cnn, y_pred_train_cnn, score_train_cnn = ev_cnn.evaluate(X_train, Y_train_single)
y_true_test_cnn, y_pred_test_cnn, score_train_cnn = ev_cnn.evaluate(X_test, Y_test_single)

### Evaluasi untuk SVM

In [ ]:
ev_svm = SVM_Evaluator(model = cls, labels = reader.labels)
y_true_train_svm, y_pred_train_svm, score_train_svm = ev_svm.evaluate(feature_train, Y_train_single)
y_true_test_svm, y_pred_test_svm, score_test_svm = ev_svm.evaluate(feature_test, Y_test_single)

# Menampilkan Confusion Matrix

In [ ]:
# Confusion Matrix
cfm = ConfusionMatrix((25, 25), 2, 2, reader.labels)
cfm.xylabel_size = 15
cfm.add(y_true_train_cnn, y_pred_train_cnn, 'Data Training CNN')
cfm.add(y_true_test_cnn, y_pred_test_cnn, 'Data Testing CNN')
cfm.add(y_true_train_svm, y_pred_train_svm, 'Data Training SVM')
cfm.add(y_true_test_svm, y_pred_test_svm, 'Data Testing SVM')
cfm.show()